In [1]:
import tensorflow as tf
from keras import layers
from keras import preprocessing
import word2vec as w2v
import dataProsessing as dp
import numpy as np
from tensorboard.plugins import projector
import os
import io
import json

In [2]:

#run this once and only once
os.chdir("../")
print(os.getcwd())

d:\desktop\MAT_180_ML_Projects\corpus-of-chords


In [4]:
datasetName = "Jsb16thSeparated(t_60_rr_re)"

corpus = dp.loadDataset("Data\\Corpi\\" + datasetName + "\\" + datasetName +".json")
vocab = dataset = json.load(open("Data\\Corpi\\" + datasetName + "\\" + datasetName +"_vocab.json"))
inv_vocab = vocab["inv_vocab"]
vocab = vocab["vocab"]



In [7]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
embedding_dim = 128
negative_samples = 4
log_dir = "logs\embeddings"

vectorizedSequences = dp.vectorizeDataset(corpus,vocab)
targets,contexts,labels = [],[],[]
for s in vectorizedSequences:
    tmp = w2v.generateTrainingData(s,vocab_size=len(vocab),window_size=1,negative_samples=negative_samples)
    targets += tmp[0]
    contexts += tmp[1]
    labels += tmp[2]
    
targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

dataset = tf.data.Dataset.from_tensor_slices(((targets,contexts),labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


word2vec = w2v.Word2Vec(len(vocab), embedding_dim,4)
word2vec.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

weights = tf.Variable(word2vec.get_layer('w2v_embedding').get_weights()[0])


checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join("data\\corpi\\" + datasetName, "embedding.ckpt"))






Epoch 1/20
64/64 [==============================] - 3s 42ms/step - loss: 1.5987 - accuracy: 0.4154
Epoch 2/20
64/64 [==============================] - 2s 38ms/step - loss: 1.5221 - accuracy: 0.9004
Epoch 3/20
64/64 [==============================] - 3s 42ms/step - loss: 1.3400 - accuracy: 0.9601
Epoch 4/20
64/64 [==============================] - 2s 38ms/step - loss: 1.1000 - accuracy: 0.9574
Epoch 5/20
64/64 [==============================] - 3s 42ms/step - loss: 0.8843 - accuracy: 0.9548
Epoch 6/20
64/64 [==============================] - 2s 38ms/step - loss: 0.7143 - accuracy: 0.9576
Epoch 7/20
64/64 [==============================] - 2s 38ms/step - loss: 0.5805 - accuracy: 0.9633
Epoch 8/20
64/64 [==============================] - 3s 39ms/step - loss: 0.4727 - accuracy: 0.9701
Epoch 9/20
64/64 [==============================] - 3s 39ms/step - loss: 0.3852 - accuracy: 0.9768
Epoch 10/20
64/64 [==============================] - 3s 41ms/step - loss: 0.3135 - accuracy: 0.9823
Epoch 11/

In [15]:
weights = np.array(weights).tolist()
print(type(weights))

with open("data\\corpi\\" + datasetName + "\\weights", "w") as outfile:
    json.dump(weights, outfile)



<class 'list'>


In [9]:
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings("data\\corpi\\" + datasetName, config)

In [ ]:

%tensorboard --logdir logs\embeddings
